<a href="https://colab.research.google.com/github/emanueleiacca/ADM-HW4/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
#movielens_20m_dataset_path = kagglehub.dataset_download('grouplens/movielens-20m-dataset')
print('Data source import complete.')


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


https://github.com/rapidsai/cudf?tab=readme-ov-file#pip

In [7]:
pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [4]:
import rmm
import cudf
rmm.reinitialize(pool_allocator=True, managed_memory=True)
print("RAPIDS cuDF is using GPU memory:", rmm.is_initialized())

RAPIDS cuDF is using GPU memory: True


In [5]:
import cupy as cp

def check_memory():
    memory_pool = cp.get_default_memory_pool()
    total_memory = cp.cuda.Device(0).mem_info[0]  # Total memory available
    used_memory = memory_pool.used_bytes()       # Memory allocated by the pool
    free_memory = total_memory - used_memory
    print(f"GPU Memory - Total: {total_memory / (1024**3):.2f} GB, Used: {used_memory / (1024**3):.2f} GB, Free: {free_memory / (1024**3):.2f} GB")

check_memory()

GPU Memory - Total: 14.65 GB, Used: 0.00 GB, Free: 14.65 GB


In [12]:
movielens_20m_dataset_path = kagglehub.dataset_download('grouplens/movielens-20m-dataset')

# Percorso ai file del dataset scaricato
dataset_directory = '/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1'

# Elenca e stampa i file presenti nella directory del dataset
for dirname, _, filenames in os.walk(dataset_directory):
    for filename in filenames:
        print(f"File trovato: {filename}")

File trovato: genome_scores.csv
File trovato: tag.csv
File trovato: movie.csv
File trovato: genome_tags.csv
File trovato: rating.csv
File trovato: link.csv


In [13]:
ratings_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/rating.csv")
links_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/link.csv")
genome_tags_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_tags.csv")
genome_scores_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/genome_scores.csv")
tags_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/tag.csv")
movies_df = cudf.read_csv("/root/.cache/kagglehub/datasets/grouplens/movielens-20m-dataset/versions/1/movie.csv")


In [14]:
check_memory()

GPU Memory - Total: 12.14 GB, Used: 0.00 GB, Free: 12.14 GB


In [15]:
print("ratings_df:",ratings_df.columns)
print("links_df:",links_df.columns)
print("genome_tags_df:",genome_tags_df.columns)
print("genome_scores_df:",genome_scores_df.columns)
print("tags_df:",tags_df.columns)
print("movies_df:",movies_df.columns)

ratings_df: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
links_df: Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')
genome_tags_df: Index(['tagId', 'tag'], dtype='object')
genome_scores_df: Index(['movieId', 'tagId', 'relevance'], dtype='object')
tags_df: Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')
movies_df: Index(['movieId', 'title', 'genres'], dtype='object')


In [16]:
import gc
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
movies_links = movies_df.merge(links_df, on="movieId", how="left")
del links_df  # Free up memory
gc.collect()
movies_links.head()

,movieId,title,genres,imdbId,tmdbId
0,1571,When the Cat's Away (Chacun cherche son chat) ...,Comedy|Romance,115856,11956
1,1572,"Contempt (Mépris, Le) (1963)",Drama,57345,266
2,1573,Face/Off (1997),Action|Crime|Drama|Thriller,119094,754
3,1574,Fall (1997),Romance,119098,37567
4,1575,Gabbeh (1996),Drama,116384,43771


In [17]:
# Merge ratings with movies
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
ratings_movies = ratings_df.merge(movies_links, on="movieId", how="left")
del ratings_df, movies_links  # Free up memory
gc.collect()
ratings_movies.head()

,userId,movieId,rating,timestamp,title,genres,imdbId,tmdbId
0,91,1777,3.5,2005-09-12 06:51:11,"Wedding Singer, The (1998)",Comedy|Romance,120888,11003
1,91,1784,4.5,2005-03-22 22:41:47,As Good as It Gets (1997),Comedy|Drama|Romance,119822,2898
2,91,1876,2.5,2005-09-12 06:56:26,Deep Impact (1998),Drama|Sci-Fi|Thriller,120647,8656
3,91,1907,3.5,2005-03-23 06:27:08,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,120762,10674
4,91,1909,3.0,2005-04-11 06:37:49,"X-Files: Fight the Future, The (1998)",Action|Crime|Mystery|Sci-Fi|Thriller,120902,846


In [18]:
rmm.reinitialize(pool_allocator=True, managed_memory=True)  # Reset GPU memory
# Merge genome scores with genome tags
genome_data = genome_scores_df.merge(genome_tags_df, on="tagId", how="left")
del genome_tags_df  # Free up memory
gc.collect()
genome_data.head()

,movieId,tagId,relevance,tag
0,3,977,0.08250,stunning
1,3,978,0.03875,stupid
2,3,979,0.07550,stupid as hell
3,3,980,0.05450,stupidity
4,3,981,0.07175,stylish


In [19]:
rmm.reinitialize(pool_allocator=True, managed_memory=True)
# Merge tags with movies
tags_movies = tags_df.merge(movies_df, on="movieId", how="left")
del tags_df  # Free up memory
gc.collect()
tags_movies.head()

,userId,movieId,tag,timestamp,title,genres
0,1741,280,courtroom setting,2007-05-12 04:05:38,Murder in the First (1995),Drama|Thriller
1,1741,288,serial killer,2007-08-18 02:59:59,Natural Born Killers (1994),Action|Crime|Thriller
2,1741,290,rape,2009-05-01 01:06:12,Once Were Warriors (1994),Crime|Drama
3,1741,293,assassin,2009-01-15 19:38:24,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
4,1741,293,assassin-in-training,2009-01-15 19:40:03,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller


In [20]:
check_memory()

GPU Memory - Total: 9.14 GB, Used: 0.00 GB, Free: 9.14 GB


In [21]:
print("ratings_movies:",ratings_movies.columns)
print("genome_data:",genome_data.columns)
print("tags_movies:",tags_movies.columns)

ratings_movies: Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres', 'imdbId',
       'tmdbId'],
      dtype='object')
genome_data: Index(['movieId', 'tagId', 'relevance', 'tag'], dtype='object')
tags_movies: Index(['userId', 'movieId', 'tag', 'timestamp', 'title', 'genres'], dtype='object')


In [22]:
ratings_movies = ratings_movies[["userId", "movieId", "rating", "title", "genres","timestamp"]]
genome_data = genome_data[["movieId", "tag", "relevance"]]
tags_movies = tags_movies[["userId", "movieId", "tag"]]
#Remove repeated columns

In [23]:
print("ratings_movies:",ratings_movies.info())
print("genome_data:",genome_data.info())
print("tags_movies:",tags_movies.info())

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   userId     int64
 1   movieId    int64
 2   rating     float64
 3   title      object
 4   genres     object
 5   timestamp  object
dtypes: float64(1), int64(2), object(3)
memory usage: 1.8+ GB
ratings_movies: None
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype
---  ------     -----
 0   movieId    int64
 1   tag        object
 2   relevance  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 335.2+ MB
genome_data: None
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 465426 entries, 0 to 465425
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userId   465426 non-null  int64
 1   movieId  465426 non-null  int64
 2   tag      465426 non-null  object
dtypes: int

In [27]:
import cudf
import pandas as pd



# Converte cuDF in pandas DataFrame
ratings_movies = ratings_movies.to_pandas()
movies = movies_df.to_pandas()

# Ora puoi usare il codice Pandas originale
ratings_movies = ratings_movies.sample(n=1000, random_state=42)

# Raggruppa i dati per utente e film (calcola la media delle valutazioni se necessario)
ratings_movies = ratings_movies.groupby(['userId', 'movieId'])['rating'].mean().reset_index()

# Creazione di un dizionario: user -> set di film visti
user_movies = ratings_movies.groupby('userId')['movieId'].apply(set).to_dict()

# Continua con il resto del codice come prima...
def generate_hash_functions(num_hashes, max_movie_id):
    a = np.random.randint(1, max_movie_id, size=num_hashes)
    b = np.random.randint(0, max_movie_id, size=num_hashes)
    c = max_movie_id + 1  # Un numero primo più grande di max_movie_id
    return [(lambda x, a=a_, b=b_, c=c: (a + x * b) % c) for a_, b_ in zip(a, b)]

# Generazione delle funzioni hash
num_hashes = 10  # Numero di funzioni hash
max_movie_id = ratings_movies["movieId"].max()
hash_functions = generate_hash_functions(num_hashes, max_movie_id)

# Calcolo della matrice delle firme
signature_matrix = {}
for user, movies in user_movies.items():
    signature = []
    for h in hash_functions:
        min_hash = min(h(movie) for movie in movies)  # Valore hash minimo per l'utente
        signature.append(min_hash)
    signature_matrix[user] = signature

# Funzione per calcolare la similarità di Jaccard tra firme
def jaccard_similarity(signature1, signature2):
    count = sum(1 for x, y in zip(signature1, signature2) if x == y)
    return count / len(signature1)

# Funzione per raccomandare film basati sulla similarità
def recommend_movies(user_id, signature_matrix, ratings, threshold=0.78):
    recommendations = []
    user_signature = signature_matrix[user_id]

    for other_user, other_signature in signature_matrix.items():
        if other_user == user_id:
            continue
        similarity = jaccard_similarity(user_signature, other_signature)
        if similarity > threshold:
            # Ottieni film valutati da altri utenti simili
            other_user_movies = set(ratings[ratings["userId"] == other_user]["movieId"])
            user_movies = set(ratings[ratings["userId"] == user_id]["movieId"])
            recommendations.extend(other_user_movies - user_movies)

    return list(set(recommendations))  # Rimuovi duplicati

# Raccomandazioni per un utente specifico
user_id = 6  # Esempio di ID utente
recommended_movies = recommend_movies(user_id, signature_matrix, ratings_movies)

# Rimuovi i film già visti dall'utente
user_watched_movies = ratings_movies[ratings_movies['userId'] == user_id]['movieId']
unseen_movies = set(recommended_movies) - set(user_watched_movies)

# Filtra i film raccomandati
film = movies[movies['movieId'].isin(unseen_movies)]

# Stampa i film consigliati
print(f"Film consigliati per l'utente {user_id} sono:", len(film))
for title in film['title']:
    print(title)

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [28]:
import numpy as np
import pandas as pd

# Carica il dataset ratings_movies


# Seleziona solo 1000 righe casuali per facilitare l'elaborazione
ratings_movies = ratings_movies.sample(n=1000, random_state=42)

# Raggruppa i dati per utente e film (calcola la media delle valutazioni se necessario)
ratings_movies = ratings_movies.groupby(['userId', 'movieId']).mean().reset_index()

# Creazione di un dizionario: user -> set di film visti
user_movies = ratings_movies.groupby('userId')['movieId'].apply(set).to_dict()

# Definizione della funzione per generare funzioni hash
def generate_hash_functions(num_hashes, max_movie_id):
    a = np.random.randint(1, max_movie_id, size=num_hashes)
    b = np.random.randint(0, max_movie_id, size=num_hashes)
    c = max_movie_id + 1  # Un numero primo più grande di max_movie_id
    return [(lambda x, a=a_, b=b_, c=c: (a + x * b) % c) for a_, b_ in zip(a, b)]

# Generazione delle funzioni hash
num_hashes = 10  # Numero di funzioni hash
max_movie_id = ratings_movies["movieId"].max()
hash_functions = generate_hash_functions(num_hashes, max_movie_id)

# Calcolo della matrice delle firme
signature_matrix = {}
for user, movies in user_movies.items():
    signature = []
    for h in hash_functions:
        min_hash = min(h(movie) for movie in movies)  # Valore hash minimo per l'utente
        signature.append(min_hash)
    signature_matrix[user] = signature

# Funzione per calcolare la similarità di Jaccard tra firme
def jaccard_similarity(signature1, signature2):
    count = sum(1 for x, y in zip(signature1, signature2) if x == y)
    return count / len(signature1)

# Funzione per raccomandare film basati sulla similarità
def recommend_movies(user_id, signature_matrix, ratings, threshold=0.78):
    recommendations = []
    user_signature = signature_matrix[user_id]

    for other_user, other_signature in signature_matrix.items():
        if other_user == user_id:
            continue
        similarity = jaccard_similarity(user_signature, other_signature)
        if similarity > threshold:
            # Ottieni film valutati da altri utenti simili
            other_user_movies = set(ratings[ratings["userId"] == other_user]["movieId"])
            user_movies = set(ratings[ratings["userId"] == user_id]["movieId"])
            recommendations.extend(other_user_movies - user_movies)

    return list(set(recommendations))  # Rimuovi duplicati

# Raccomandazioni per un utente specifico
user_id = 6  # Esempio di ID utente
recommended_movies = recommend_movies(user_id, signature_matrix, ratings_movies)

# Rimuovi i film già visti dall'utente
user_watched_movies = ratings_movies[ratings_movies['userId'] == user_id]['movieId']
unseen_movies = set(recommended_movies) - set(user_watched_movies)

# Filtra i film raccomandati
movies = pd.read_csv('movies.csv')  # Assicurati di avere il dataset con i dettagli dei film
film = movies[movies['movieId'].isin(unseen_movies)]

# Stampa i film consigliati
print(f"Film consigliati per l'utente {user_id} sono:", len(film))
for title in film['title']:
    print(title)


TypeError: agg function failed [how->mean,dtype->object]